In [ ]:
import torch
import random
from PIL import Image, ImageFilter
from IPython.display import clear_output, display
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
negative_prompt = "ugly, deformed, bad anatomy, bad face, bad eyes, bad lips, distorted hands, asymmetrical face, uneven features, blurry, missing details, oversaturated, undersaturated, unnatural colors, bad proportions, extra limbs, missing limbs, unnatural poses, extra fingers, missing fingers, mutated hands, disconnected limbs, broken limbs, incoherent background, pixelated, overexposed, poor composition, awkward angles, flat shading, dull colors, mismatched textures, text artifacts, watermarks, grainy, fuzzy, washed out, overly bright, too dark, uneven lighting, unrealistic shadows, bad focus, incomplete objects, stretched textures, clashing colors, jagged lines, blurred edges, warped objects, inconsistent physics, floating objects, poor depth, unbalanced composition, misaligned objects, incorrect scaling, bad transitions, overly sharpened details, chromatic aberrations, texture stretching, rigid poses, clipping artifacts, extreme noise."
prompts = [
    "pixelhue, anime style, cinematic lighting, realistic reflections, volumetric light beams, tropical tones, dramatic shadows, hyper-detailed textures, atmospheric perspective, ultra-realistic environments, tropical beach textures, vibrant colors, a tropical paradise with white sand beaches and crystal-clear water, relaxing environment, peaceful atmosphere, vibrant reflections, luminous sky, panoramic vista, palm trees swaying in the breeze at sunset.",
    "pixelhue, anime style, cinematic lighting, realistic reflections, volumetric light beams, winter tones, dramatic shadows, hyper-detailed textures, atmospheric perspective, ultra-realistic environments, snowy village textures, vibrant colors, a snowy mountain village with cozy cabins and snow-covered trees, cozy environment, peaceful atmosphere, vibrant reflections, luminous sky, panoramic vista, soft snowfall at dusk.",
    "pixelhue, anime style, cinematic lighting, realistic reflections, volumetric light beams, extraterrestrial tones, dramatic shadows, hyper-detailed textures, atmospheric perspective, ultra-realistic environments, alien planet textures, vibrant colors, an alien planet with strange flora and fauna, otherworldly environment, mysterious atmosphere, vibrant reflections, luminous sky, panoramic vista, double sunset.",
]
with torch.autocast("cuda"):
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", cache_dir="models")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", cache_dir="models")
    pipeLine = StableDiffusionPipeline.from_single_file("models/animefull-final-pruned-fp16.safetensors", text_encoder=text_encoder, torch_dtype=torch.float16, tokenizer=tokenizer, cache_dir="models")
    pipeLine.scheduler = DPMSolverMultistepScheduler.from_config(pipeLine.scheduler.config, use_karras_sigmas=True)
    pipeLine.load_lora_weights("models/lora", weight_name="pixelhue.safetensors", weight=1.0)
    pipeLine.enable_model_cpu_offload()
    pipeLine.enable_vae_slicing()
    clear_output()
    for prompt in prompts:
        generator = torch.Generator(device="cuda").manual_seed(0)
        inference = pipeLine(eta=0.0, height=576, width=1024, prompt=prompt, clip_skip=2, latents=None, add_noise=True, temperature=1.0, return_dict=True, output_type="pil", denoising_end=0.2, guidance_scale=7.5, 
        denoising_start=0.8, generator=generator, guidance_rescale=0.7, num_inference_steps=20, num_images_per_prompt=1, max_embeddings_multiples=3, negative_prompt=negative_prompt).images
        for image in inference:
            display(image.filter(ImageFilter.SHARPEN))